In [1]:
%%capture
!pip install captcha

In [2]:
import torch
from diffusers import StableDiffusionPipeline
import os
from torchvision import transforms
from PIL import Image as PilImage
from captcha.image import ImageCaptcha
import random
import string
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Image as dispImage
import cv2

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
## Paths
os.makedirs('/content/drive/My Drive/EE 5561 Course Project', exist_ok=True)
root_path = '/content/drive/My Drive/EE 5561 Course Project'
font_path = '/content/drive/My Drive/EE 5561 Course Project/fonts/Typography times.ttf'

captcha_dataset_path = os.path.join(root_path, "new_captcha_dataset")
high_res_background_path = os.path.join(root_path, "new_high_res_background")
captcha_without_noise = os.path.join(root_path, "captcha_without_noise")

os.makedirs(captcha_dataset_path, exist_ok=True)
os.makedirs(high_res_background_path, exist_ok=True)
os.makedirs(captcha_without_noise, exist_ok=True)

In [1]:
# root_path = os.getcwd()
# font_path = os.path.join(root_path, 'fonts/Typography times.ttf')
# captcha_dataset_path = os.path.join(root_path, "new_captcha_dataset")
# high_res_background_path = os.path.join(root_path, "new_high_res_background")
# captcha_without_noise = os.path.join(root_path, "captcha_without_noise")

NameError: name 'os' is not defined

In [ ]:
# Prompts for CIFAR-10 classes
cifar10_prompts = [
    "photo of an airplane",
    "photo of an automobile",
    "photo of a bird",
    "photo of a cat",
    "photo of a deer",
    "photo of a dog",
    "photo of a frog",
    "photo of a horse",
    "photo of a ship",
    "photo of a truck"
]

In [ ]:
%%capture
# Load the pre-trained Stable Diffusion model
model = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4").to("cuda")

In [ ]:
def generate_captcha_dataset_without_noise(num_images, output_path, width=256, height=200):
    """
    Generates CAPTCHA images without noise using the captcha library.

    Args:
        num_images (int): Number of CAPTCHA images to generate.
        output_path (str): Directory to save the generated CAPTCHA images.
        width (int): Width of the CAPTCHA image.
        height (int): Height of the CAPTCHA image.

    """

    captcha_texts = []

    for i in range(num_images):
        # Random seed for reproducibility
        random.seed(5561 + i)

        # Randomly decide the length of the CAPTCHA
        length = random.choice([4, 5])

        # Adjust font size based on the CAPTCHA length
        font_size = 130 if length == 4 else 100

        # Initialize the CAPTCHA generator with custom configurations
        image_captcha = ImageCaptcha(
            width=width,
            height=height,
            fonts=[font_path],
            font_sizes=[font_size]
        )
        image_captcha.character_rotate = (-20, 20)  # Rotate characters between -30° and 30°
        image_captcha.character_offset_dx = (10, 20)  # Horizontal offset for characters
        image_captcha.character_offset_dy = (10, 20)  # Vertical offset for characters
        image_captcha.character_warp_dx = (0, 0.3)  # Horizontal warping
        image_captcha.character_warp_dy = (0, 0.3)  # Vertical warping
        image_captcha.word_space_probability = 0.1  # Probability of adding extra spacing

        # Generate random CAPTCHA text
        text = ''.join(random.choices(string.ascii_letters + string.digits, k=length))

        # Generate the CAPTCHA image
        captcha_image = image_captcha.generate_image(text).convert("RGB")

        file_path = os.path.join(output_path, f"{text}.png")
        captcha_image.save(file_path)

        # Append text and file path to the list
        captcha_texts.append((text, file_path))

    return captcha_texts


In [ ]:
def generate_captcha_dataset_with_noise(num_images, output_path, width=256, height=200, total_noise=5000):
    """
    Generates CAPTCHA images with noisy backgrounds using custom configurations.

    Args:
        num_images (int): Number of CAPTCHA images to generate.
        output_path (str): Directory to save the generated CAPTCHA images.
        width (int): Width of the CAPTCHA image.
        height (int): Height of the CAPTCHA image.
        total_noise (int): Adds a fixed amount of noise to each image of the magnitude: 15 or 16.

    Returns:
        List[Tuple(str, str)]: List of tuples containing the CAPTCHA text and its file path.
    """

    captcha_texts = []

    for i in range(num_images):
        # Random seed for reproducibility
        random.seed(5561 + i)

        # Randomly decide the length of the CAPTCHA
        length = random.choice([4, 5])

        # Adjust font size based on the CAPTCHA length
        font_size = 130 if length == 4 else 100

        # Initialize the CAPTCHA generator with custom configurations
        image_captcha = ImageCaptcha(
            width=width,
            height=height,
            fonts=[font_path],
            font_sizes=[font_size]
        )
        image_captcha.character_rotate = (-20, 20)  # Rotate characters between -30° and 30°
        image_captcha.character_offset_dx = (10, 20)  # Horizontal offset for characters
        image_captcha.character_offset_dy = (10, 20)  # Vertical offset for characters
        image_captcha.character_warp_dx = (0, 0.3)  # Horizontal warping
        image_captcha.character_warp_dy = (0, 0.3)  # Vertical warping
        image_captcha.word_space_probability = 0.1  # Probability of adding extra spacing

        # Generates random CAPTCHA text
        text = ''.join(random.choices(string.ascii_letters + string.digits, k=length))

        # Generates the CAPTCHA image
        captcha_image = image_captcha.generate_image(text).convert("RGB")

        # Converting the image to a NumPy array for noise manipulation
        captcha_array = np.array(captcha_image)

        noise = np.zeros_like(captcha_array)
        indices = np.random.choice(np.arange(captcha_array.size), total_noise, replace=False)
        noise.flat[indices] = np.random.choice([-1, 1], size=total_noise) * np.random.randint(15, 17)
        noisy_array = np.clip(captcha_array + noise, 0, 255).astype('uint8')

        # converting back to a PIL image
        noisy_image = PilImage.fromarray(noisy_array)

        # saving path
        file_path = os.path.join(output_path, f"{text}.png")
        noisy_image.save(file_path)

        captcha_texts.append((text, file_path))

    return captcha_texts

In [ ]:
def generate_high_res_background(num_images):
    """
    Generates high-resolution background images and returns them with their class labels.

    Args:
        num_images (int): Number of high-resolution background images to generate.

    Returns:
        List[Tuple(str, PIL.Image.Image)]: A list of tuples, where each tuple contains the last word of the class prompt
                                           and the corresponding generated image.
    """
    background_images = []
    for i in range(num_images):
        # Randomly select a CIFAR-10 class prompt
        class_prompt = cifar10_prompts[torch.randint(0, len(cifar10_prompts), (1,)).item()]

        # Extract the last word from the class prompt (e.g., "airplane", "cat")
        object_name = class_prompt.split()[-1]

        # Generate a high-resolution image
        with torch.no_grad():
            generated_image = model(prompt=class_prompt, num_inference_steps=50).images[0]

        # Append the tuple (object_name, generated_image) to the list
        background_images.append((object_name, generated_image))

    return background_images

In [ ]:
# Number of images to generate
num_noisy_captchas = 1000 # CAPTCHA images with noise
num_captchas_without_noise = 5000 # CAPTCHA images without noise
num_backgrounds = 100 # high-res generated object images without noise

In [ ]:
# Generating CAPTCHA images without noise for model training
generate_captcha_dataset_without_noise(num_captchas_without_noise, output_path=captcha_without_noise);

In [ ]:
# Generating CAPTCHA text images
captcha_texts = generate_captcha_dataset_with_noise(num_noisy_captchas, output_path=captcha_dataset_path)

In [ ]:
background_images = generate_high_res_background(num_backgrounds)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
# Saving object/background images where the filenames have class-wise indexing,
# e.g., "cat_0", "cat_1"... and "dog_0", "dog_1"... and so on.

# Dictionary to keep track of the count for each class
class_counters = {}

# iterating over the background_images list
for label, image in background_images:
    # Initialize the counter for the class if not already present
    if label not in class_counters:
        class_counters[label] = 1

    # generating filename with the class name and counter
    filename = f"{label}_{class_counters[label]}.png"
    file_path = os.path.join(high_res_background_path, filename)

    # Saving image
    image.save(file_path)

    # counter increment for the class
    class_counters[label] += 1

In [ ]:
 # torch.cuda.empty_cache()

In [ ]:
# image_captcha = ImageCaptcha(
#             width=width,
#             height=height,
#             fonts=[font_path],
#             font_sizes=[font_size]
#         )
# image_captcha.character_rotate = (-20, 20)  # Rotate characters between -30° and 30°
# image_captcha.character_offset_dx = (10, 20)  # Horizontal offset for characters
# image_captcha.character_offset_dy = (10, 20)  # Vertical offset for characters
# image_captcha.character_warp_dx = (0, 0.3)  # Horizontal warping
# image_captcha.character_warp_dy = (0, 0.3)  # Vertical warping
# image_captcha.word_space_probability = 0.1  # Probability of adding extra spacing

# # Generates random CAPTCHA text
# text = ''.join(random.choices(string.ascii_letters + string.digits, k=length))

# # Generates the CAPTCHA image
# captcha_image = image_captcha.generate_image(text).convert("RGB")

# # Converting the image to a NumPy array for noise manipulation
# captcha_array = np.array(captcha_image)

# noise = np.zeros_like(captcha_array)
# indices = np.random.choice(np.arange(captcha_array.size), total_noise, replace=False)
# noise.flat[indices] = np.random.choice([-1, 1], size=total_noise) * np.random.randint(15, 17)
# noisy_array = np.clip(captcha_array + noise, 0, 255).astype('uint8')